In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from travis_attack.utils import round_t

tokenizer = AutoTokenizer.from_pretrained("textattack/albert-base-v2-CoLA")

model = AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-CoLA")

In [ ]:

text =['however however tempered elements are the direct antithesis of what is right. ',
 "though mild amusing I can't recommend it",
 'this is a film about film living in its own head',
 'although small movie impact ',
 'even tunney misfire though',
 'although somewhat inane cross between xxx and vertical limits',
 'Thoughthoughhill may not enough to salvage this boxing movie',
 'though though stunning film tour de force.',
 'although however Diaz wears her welcome in her performance',
 'though though just collection of this and that fills time without unified whole',
 'depths of despair believable',
 'though who needs love',
 'although however the repread of material already plumbed by martin scorsese',
 'I thought the watch stopped keeping time as I logged clockstoppers',
 'Epps neither charisma nor the natural affability that made tucker star',
 "sometimes but because simone is not real she can't provide conflict.",
 'the sweet home stereotype',
 'though though plodding teen remake mechanical grease on the plot twists',
 'however sadness though not without fun',
 'though though delightful coming-of-age story',
 'half baked',
 "movies to you misty even when you don't want",
 'though though stiff and schmaltzy directed',
 'however objects could go long',
 'but provoke sexual romp has something',
 'maryam now timely',
 'however although Ponski found perfect material to address his own World War iI experience in his Signature Style addressed',
 'though though many shallower movies these days are shorter',
 'it expose the hustler life of male hustlers exploitive without insightful',
 "other than that the cast list eye see you junk isn't fooled",
 'although however you must-see list ecks',
 'though though the fast runner transports the viewer into space',
 'sometimes the movie idiomatic lackluster',
 'comedy bonehead summer.',
 'even though poor editing bad ultracheesy dialogue about radical action',
 'this version justice stevenson both sci-fi genre',
 'when i see madonna.',
 'she is stylishly directed with verve.',
 'singles culture one of the best',
 'even if 88 minute highlight reel is longer',
 'victim domestic abuse victims pity and terror.',
 'although however magical the film not in mysterious spring performances',
 'character portrait romantic comedy beat-the-clock thriller',
 "eventhough Kusk's brillant here",
 'though though fine movie go see it ',
 'however it claims simple diversion for kids',
 'cinema movies movies movies',
 'though though superior genre storytelling gets under our skin simply crossing the nuclear line',
 'although however corpuse count ultimately overrides what little we learn about vicarious redemption',
 'though though interesting unfaithfully']
#text = ["hello my name is tom", "I liked this movie", "I really did enjoy this"]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
#mylabels = [1,1,1,1,1,1,1,1,1,1]

In [ ]:
encoding = tokenizer(text, padding=True, return_tensors="pt").to('cuda')
#input_ids, attention_mask = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

In [ ]:
cola_model = model 
cola_tokenizer = tokenizer
def get_cola_probs(pp_l, cfg, cola_tokenizer, cola_model): 
    inputs = cola_tokenizer(pp_l, return_tensors="pt", padding=True, truncation=True).to(cfg.device)
    with torch.no_grad():
        logits = cola_model(**inputs).logits
        probs = logits.softmax(1)
    return probs 

In [ ]:
model_output = model(**encoding)
probs = torch.softmax(model_output.logits,1)

for p,t in zip(probs,text): 
    print(round_t(p), t)

[0.83 0.17] however however tempered elements are the direct antithesis of what is right. 
[0.34 0.66] though mild amusing I can't recommend it
[0.03 0.97] this is a film about film living in its own head
[0.95 0.05] although small movie impact 
[0.35 0.65] even tunney misfire though
[0.71 0.29] although somewhat inane cross between xxx and vertical limits
[0.97 0.03] Thoughthoughhill may not enough to salvage this boxing movie
[0.97 0.03] though though stunning film tour de force.
[0.93 0.07] although however Diaz wears her welcome in her performance
[0.78 0.22] though though just collection of this and that fills time without unified whole
[0.31 0.69] depths of despair believable
[0.64 0.36] though who needs love
[0.94 0.06] although however the repread of material already plumbed by martin scorsese
[0.56 0.44] I thought the watch stopped keeping time as I logged clockstoppers
[0.85 0.15] Epps neither charisma nor the natural affability that made tucker star
[0.57 0.43] sometimes but

Impressions: 
    * first random batch of 10, me and the model agreed with 9.
    * second random batch of 10, me and the model agreed on all (though they were mostly all bad)
    * third batch, agreed with 9/10
    * fourth batch from simple dataset, we agreed on all. I'm fine with random punctuation and the model seems to be okay with it for the most part. 
    * fifth batch, we agreed on 8/10. 
    * sixth batch, i put in a bunch of original examples. it agreed with 9/10, then 10/10
    * finally I made a bunch of paraphrases myself. It liked 9/10. 
    
The best thing about it is that it gets rid of the absolute trash. Sometimes it seems to put a valid sentence as invalid but overall it does pretty well. 
   
I think we can go with this and set the cutoff maybe to a class 0 rating ( acceptable) of 0.5 - so if we get higher than this probability for "crap" then we ditch it. keep this the same way round as sts score. 
cola_acceptable_threshold = 0.5
   
   
maybe this is hard for shorter sentences? 